# Prepare the final dataset (X_train,y_train)

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import f1_score, classification_report
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
print(lgb.__version__)
from sklearn.ensemble import RandomForestClassifier

4.5.0


In [2]:
df=pd.read_csv("X_train.csv")
label_to_num = {'Very Low': 0, 'Low': 1, 'Average': 2, 'High': 3, 'Very High': 4}
df['piezo_groundwater_level_category'] = df['piezo_groundwater_level_category'].map(label_to_num)
y_train=df['piezo_groundwater_level_category']
# df.fillna(-1, inplace=True)

In [3]:
df.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [4]:
# df_search=df.drop(columns=['piezo_station_pe_label', 'piezo_measurement_date',"piezo_groundwater_level_category","piezo_station_bss_id"])
df_search = df.drop(columns=['piezo_groundwater_level_category'])
# df_filtered = df_search[df_search['month'].between(4, 9)]

In [5]:
df_search = df.drop(columns=['Unnamed: 0'])

In [6]:
df_search = df_search.drop(columns=['piezo_groundwater_level_category'])
df_search.head(10)

,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_station_investigation_depth,meteo_rain_height,meteo_temperature_avg,hydro_observation_result_elab,day,month,year
0,232.00,5.748241,45.828148,20.00,0.2,0.7,61.100000,1,1,2020
1,247.25,5.356637,46.028102,35.60,0.0,-1.2,3112.966667,1,1,2020
2,218.77,5.220795,45.895734,35.22,NaN,-0.6,3607.900000,1,1,2020
3,499.85,5.948977,46.201180,34.20,0.0,0.4,10182.133333,1,1,2020
4,260.00,5.313353,46.136402,37.30,0.0,-0.8,2.833333,1,1,2020
5,250.00,5.679690,45.874738,50.00,0.0,3.5,61.100000,1,1,2020
6,208.21,5.285156,45.837397,21.00,0.0,-1.2,3607.900000,1,1,2020
7,558.68,6.069710,46.329030,78.00,0.0,0.5,20.500000,1,1,2020
8,250.45,5.823274,45.919630,10.00,0.0,-0.4,14373.466667,1,1,2020
9,265.22,4.830754,46.018341,62.60,0.0,-0.3,10.866667,1,1,2020


In [7]:
X_train, X_val, y_train, y_val = train_test_split(df_search, y_train, test_size=0.2, random_state=42)

In [8]:
xgb_model = xgb.XGBClassifier(device='cuda', random_state=42)
lgb_model = lgb.LGBMClassifier(device='cuda',random_state=42)
rf_model = RandomForestClassifier(random_state=0)

In [100]:
param_distributions = {
    'n_estimators': [75, 100, 125],
    'max_depth': [11, 12, 13],
    'learning_rate': [0.1, 0.2, 0.3],
}
f1_scorer = make_scorer(f1_score,average='weighted')
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Nombre d'itérations pour la recherche
    scoring=f1_scorer,
    cv=3,  # Validation croisée à 3 folds
    verbose=3,
    random_state=42)

# Exécution de RandomizedSearchCV
random_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres trouvés
print("Meilleurs paramètres :", random_search.best_params_)



Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: 

In [9]:
import xgboost as xgb
from xgboost.callback import TrainingCallback

params = { 
    "tree_method" : "hist", 
    "device" : "cuda",  
    "objective": "multi:softmax",  
    "max_depth": 12,
    "learning_rate": 0.25,
    "n_estimators": 300,
    "verbosity": 1,
    'sub_sample': 0.6
}

class EarlyStoppingF1(TrainingCallback):
    def __init__(self, X_val, y_val, rounds=10, best_score=-float('inf')):
        self.X_val = X_val
        self.y_val = y_val
        self.rounds = rounds 
        self.best_score = best_score
        self.no_improvement_rounds = 0  

    def after_iteration(self, model, epoch, evals_log):
        """Calculer le F1-score après chaque itération et vérifier l'amélioration"""
        preds = model.predict(xgb.DMatrix(self.X_val))
        preds = preds.astype(int) 
        f1 = f1_score(self.y_val, preds, average="macro")  # F1-score macro

        print(f"[Epoch {epoch}] F1-Score (Validation): {f1:.4f}")
        if f1 > self.best_score:
            self.best_score = f1
            self.no_improvement_rounds = 0  # Reset counter
        else:
            self.no_improvement_rounds += 1

        # Si le nombre de rounds sans amélioration dépasse le seuil, arrêter l'entraînement
        if self.no_improvement_rounds > self.rounds:
            print(f"Early stopping after {epoch} rounds with no improvement in F1-score.")
            return True 
        return False  

# Initialiser le callback EarlyStopping basé sur F1-score
early_stop_f1 = EarlyStoppingF1(X_val, y_val, rounds=10)

class SaveEveryNEpochs(TrainingCallback):
    def __init__(self, save_interval, save_path):
        self.save_interval = save_interval
        self.save_path = save_path

    def after_iteration(self, model, epoch, evals_log):
        if (epoch + 1) % self.save_interval == 0:
            save_file = f"{self.save_path}_epoch_{epoch + 1}.json"
            model.save_model(save_file)
            print(f"Model saved at epoch {epoch + 1} to {save_file}")
        return False  # Return False to continue training

# Instancier le callback
save_callback = SaveEveryNEpochs(save_interval=10, save_path="xgboost_model")

# Initialiser le classifieur
model = xgb.XGBClassifier(**params,
                          early_stopping_rounds=10,
                          callbacks=[early_stop_f1,save_callback])

In [10]:
model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

In [119]:
model.save_model("best_model.json")

In [120]:
loaded_model = xgb.Booster()
loaded_model.load_model("best_model.json")
dval= xgb.DMatrix(X_val)

In [121]:
y_pred = loaded_model.predict(dval)
f1 = f1_score(y_val, y_pred,average="weighted")
print(f"\nFinal F1 Score on Test Set: {f1:.2f}")

# Rapport complet
print("\nClassification Report:")
print(classification_report(y_val, y_pred))


Final F1 Score on Test Set: 0.91

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96    106904
           1       0.91      0.91      0.91    124968
           2       0.88      0.88      0.88    128719
           3       0.89      0.89      0.89    115883
           4       0.95      0.94      0.94     85940

    accuracy                           0.91    562414
   macro avg       0.92      0.92      0.92    562414
weighted avg       0.91      0.91      0.91    562414



# Submission

In [ ]:
df_test=pd.read_csv("X_test.csv")
rows_id=pd.read_csv("y_test_submission_example_Hi5.csv")
rows_id=rows_id["row_index"]
print(df_test.shape)

(611208, 12)


In [123]:
df_test.head(10)
df_test = df_test.drop(columns = ['piezo_qualification', 'Unnamed: 0'])

In [124]:
dtest = xgb.DMatrix(df_test)
y_pred = loaded_model.predict(dtest)
print(len(y_pred))
num_to_label= {0:'Very Low', 1:'Low', 2:'Average', 3:'High', 4:'Very High'}
y_pred_classes = y_pred.argmax(axis=1) if len(y_pred.shape) > 1 else y_pred
y_pred_labels = [num_to_label[int(label)] for label in y_pred_classes]
pred_df = pd.DataFrame({"row_index": rows_id, "piezo_groundwater_level_category": y_pred_labels})
pred_df.to_csv('eigth_submission.csv',index=False)

611208
